In [ ]:
import numpy as np
from numpy.random import RandomState
prng = RandomState(20150101)
import timeit
from vispy import app, gloo

In [ ]:
app.use_app('ipynb_webgl')

In [ ]:
vertex_code = """
attribute vec4 a_position;
void main () {
    gl_Position = a_position;
    gl_PointSize = 1.;
}
"""

fragment_code = """
void main()
{
    gl_FragColor = vec4(1,1,1,1);
}
"""

In [ ]:
class Canvas(app.Canvas):
    def __init__(self):
        app.Canvas.__init__(self, size=(400, 400), keys='interactive')
        #self.timer = app.Timer(1, self.on_timer, start=True)
        #self.times = []

    def on_initialize(self, event):
        self.program = gloo.Program(vertex_code, fragment_code)

        n = 1000000
        self.data = np.zeros((n, 4), dtype=np.float32)
        self.data[:, :2] = .15 * prng.randn(n, 2)
        self.data[:, 3] = 1.
        self.program['a_position'] = self.data

        gloo.clear((0, 0, 0, 1))

    def on_draw(self, event):
        gloo.clear()
        self.program.draw('points')

    def on_resize(self, event):
        gloo.set_viewport(0, 0, *event.size)

In [ ]:
c = Canvas()
c.show()

In [ ]:
%%javascript
// TODO: register frame number
var canvas = vispy._canvases[0];
canvas.on_paint(function (e) {
    // Finished painting.
    canvas.gl.flush();
    canvas.gl.finish();
    console.timeEnd('paint');
});
var i = 0;
window._timer = window.setInterval(function () {
    console.time('paint');
    canvas.update();
}, 1000/60.0*20);

In [ ]:
%%javascript
window.clearInterval(window._timer)